In [ ]:
import requests

def get_access_token():
    url = "https://api.swiss-ski.ch/auth/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "password",
        "client_id": "rest.force8.coach",
        "client_secret": "24A6bb_FBxMfc7",
        "username": "marc.guerber@swiss-ski.ch",
        "password": "EAuztqBLj4UAu82"
    }
    response = requests.post(url, headers=headers, data=data)
    return response.json()

def make_api_request(access_token):
    url = "https://api.swiss-ski.ch/api/rest/request/get/club_athlete_details"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "id_club": "6",
        "page": "1"
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()


# Example usage:
token_response = get_access_token()
if 'access_token' in token_response:
    api_response = make_api_request(token_response['access_token'])
    # Loop through each entry and print 'first_name'
    for athlete in api_response:  # Assuming api_response is directly the list
        print("First Name:", athlete.get('first_name'))
else:
    print("Failed to obtain access token:", token_response)



Failed to obtain access token: {'error': 'invalid_grant', 'error_description': 'Invalid username and password combination'}
